In [24]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [25]:
HTML('<script src="lib/d3/d3.min.js"></script>')

In [26]:
css_text = '''
body {
  font: 10px sans-serif;
}

.chord path {
  fill-opacity: .67;
  stroke: #F00;
  stroke-width: .5px;
}

'''

In [29]:
js_text_template = Template('''
var matrix = [
  [11975,  5871, 8916, 2868],
  [ 1951, 10048, 2060, 6171],
  [ 8010, 16145, 8090, 8045],
  [ 1013,   990,  940, 6907]
];

var matrix = [[66706, 13217, 5695, 3473, 2291, 1486, 1046, 918, 659, 587, 454, 409, 393, 383, 251, 232, 218, 176, 189, 140, 129, 107, 109, 120, 77, 88, 77, 81, 57, 55], [6548, 3211, 1668, 1151, 739, 638, 362, 309, 253, 220, 136, 166, 101, 88, 78, 66, 55, 53, 60, 51, 58, 41, 38, 46, 32, 27, 16, 21, 13, 24], [3012, 1889, 1175, 680, 590, 457, 299, 241, 232, 171, 138, 122, 118, 94, 88, 64, 77, 50, 65, 38, 32, 38, 49, 29, 24, 27, 13, 13, 22, 18], [2034, 1196, 768, 549, 372, 302, 254, 211, 169, 155, 99, 93, 102, 79, 65, 57, 33, 72, 35, 48, 37, 32, 32, 22, 18, 31, 19, 24, 14, 15], [1233, 825, 548, 425, 333, 216, 162, 163, 125, 114, 128, 86, 82, 61, 60, 58, 49, 44, 49, 44, 31, 27, 37, 27, 24, 26, 26, 6, 15, 9], [825, 604, 385, 276, 232, 190, 155, 131, 114, 103, 48, 53, 41, 38, 62, 58, 33, 32, 36, 36, 34, 35, 19, 19, 19, 10, 16, 15, 7, 3], [570, 409, 306, 220, 180, 145, 132, 85, 82, 68, 74, 55, 79, 40, 33, 27, 25, 31, 23, 30, 20, 25, 24, 11, 24, 12, 9, 10, 6, 10], [517, 380, 252, 152, 163, 164, 102, 109, 46, 46, 55, 42, 39, 32, 39, 27, 20, 29, 27, 28, 12, 12, 12, 8, 17, 7, 3, 9, 8, 7], [387, 315, 179, 134, 111, 91, 82, 91, 67, 30, 35, 27, 33, 28, 28, 16, 13, 17, 10, 13, 3, 14, 9, 5, 13, 5, 7, 9, 5, 5], [341, 199, 160, 149, 98, 95, 82, 41, 41, 25, 44, 37, 32, 29, 21, 25, 8, 22, 13, 13, 10, 15, 10, 14, 7, 4, 1, 3, 1, 2], [260, 182, 127, 138, 79, 67, 62, 43, 33, 35, 31, 25, 21, 18, 28, 11, 13, 25, 24, 11, 8, 9, 6, 9, 3, 6, 6, 6, 2, 1], [195, 160, 106, 85, 93, 59, 50, 42, 21, 34, 12, 12, 31, 23, 19, 16, 24, 11, 6, 22, 8, 6, 11, 8, 3, 3, 4, 2, 1, 4], [183, 124, 109, 61, 44, 60, 29, 43, 32, 14, 16, 27, 28, 19, 9, 8, 8, 3, 8, 4, 7, 5, 4, 9, 3, 3, 2, 6, 1, 1], [144, 119, 100, 70, 47, 29, 37, 31, 40, 30, 12, 14, 16, 7, 7, 18, 3, 9, 3, 7, 7, 4, 1, 0, 1, 2, 5, 4, 1, 1], [180, 98, 79, 54, 37, 33, 24, 26, 23, 19, 24, 16, 14, 28, 9, 10, 6, 7, 6, 6, 6, 6, 1, 2, 0, 0, 6, 0, 7, 1], [114, 99, 61, 56, 33, 25, 16, 29, 13, 20, 14, 19, 16, 6, 7, 9, 9, 4, 12, 7, 5, 1, 2, 4, 0, 1, 4, 1, 0, 1], [105, 75, 61, 37, 40, 31, 22, 23, 11, 21, 16, 11, 6, 10, 14, 13, 1, 1, 2, 4, 2, 0, 1, 5, 1, 3, 2, 0, 0, 0], [102, 72, 42, 39, 28, 24, 24, 23, 8, 13, 9, 4, 11, 10, 8, 8, 5, 6, 2, 2, 3, 1, 0, 1, 0, 1, 0, 0, 0, 0], [80, 43, 50, 21, 35, 17, 24, 18, 9, 12, 5, 14, 2, 6, 8, 6, 5, 1, 1, 4, 5, 0, 2, 4, 1, 1, 0, 1, 1, 0], [55, 62, 39, 25, 33, 20, 26, 18, 7, 10, 10, 5, 9, 3, 8, 1, 7, 6, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1], [77, 59, 26, 26, 29, 25, 9, 17, 9, 4, 4, 6, 7, 7, 4, 6, 3, 7, 6, 2, 2, 1, 0, 3, 0, 0, 2, 0, 0, 0], [39, 47, 48, 19, 19, 33, 22, 13, 5, 2, 14, 1, 2, 5, 0, 1, 1, 3, 0, 3, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0], [71, 33, 27, 18, 24, 10, 14, 7, 3, 11, 4, 2, 3, 4, 1, 3, 1, 0, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], [52, 35, 29, 16, 18, 19, 15, 7, 3, 6, 6, 10, 6, 3, 1, 2, 7, 1, 4, 3, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0], [37, 28, 25, 19, 20, 8, 7, 10, 2, 8, 5, 2, 9, 4, 0, 2, 0, 1, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], [64, 30, 40, 8, 10, 3, 13, 3, 3, 12, 2, 3, 0, 4, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [50, 31, 16, 8, 6, 7, 3, 10, 6, 5, 2, 2, 4, 4, 0, 3, 4, 3, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [35, 15, 11, 4, 8, 5, 13, 9, 1, 3, 3, 4, 4, 2, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [45, 15, 13, 8, 6, 8, 11, 4, 1, 1, 0, 0, 2, 3, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [19, 14, 18, 5, 8, 10, 6, 6, 3, 6, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


var chord = d3.layout.chord()
    .padding(.05)
    .sortSubgroups(d3.descending)
    .matrix(matrix);

var width = 700,
    height = 700,
    innerRadius = Math.min(width, height) * .41,
    outerRadius = innerRadius * 1.1;

var fill = d3.scale.ordinal()
    .domain(d3.range(4))
    .range(["#0040ff", "#ff8000", "#ffbf00", "#ffff00", "#bfff00", "#ff0000", "#00ffff", "#0040ff"]);
//var svg = d3.select("body").append("svg")
var svg = d3.select("#$graphdiv").append("svg")
    .attr("width", width)
    .attr("height", height)
  .append("g")
    .attr("transform", "translate(" + width / 2 + "," + height / 2 + ")");

svg.append("g").selectAll("path")
    .data(chord.groups)
  .enter().append("path")
    .style("fill", function(d) { return fill(d.index); })
    .style("stroke", function(d) { return fill(d.index); })
    .attr("d", d3.svg.arc().innerRadius(innerRadius).outerRadius(outerRadius))
    .on("mouseover", fade(.1))
    .on("mouseout", fade(1));

var ticks = svg.append("g").selectAll("g")
    .data(chord.groups)
  .enter().append("g").selectAll("g")
    .data(groupTicks)
  .enter().append("g")
    .attr("transform", function(d) {
      return "rotate(" + (d.angle * 180 / Math.PI - 90) + ")"
          + "translate(" + outerRadius + ",0)";
    });

ticks.append("line")
    .attr("x1", 1)
    .attr("y1", 0)
    .attr("x2", 5)
    .attr("y2", 0)
    .style("stroke", "#000");

ticks.append("text")
    .attr("x", 8)
    .attr("dy", ".35em")
    .attr("transform", function(d) { return d.angle > Math.PI ? "rotate(180)translate(-16)" : null; })
    .style("text-anchor", function(d) { return d.angle > Math.PI ? "end" : null; })
    .text(function(d) { return d.label; });

svg.append("g")
    .attr("class", "chord")
  .selectAll("path")
    .data(chord.chords)
  .enter().append("path")
    .attr("d", d3.svg.chord().radius(innerRadius))
    .style("fill", function(d) { return fill(d.target.index); })
    .style("opacity", 1)
    .append("title")
        .text(function(d) { return "N"; });

// Returns an array of tick angles and labels, given a group.
function groupTicks(d) {
  var k = (d.endAngle - d.startAngle) / d.value;
  return d3.range(0, d.value, 1000).map(function(v, i) {
    return {
      angle: v * k + d.startAngle,
      label: i % 5 ? null : v / 1000 + "k"
    };
  });
}

// Returns an event handler for fading a given chord group.
function fade(opacity) {
  return function(g, i) {
    svg.selectAll(".chord path")
        .filter(function(d) { return d.source.index != i && d.target.index != i; })
      .transition()
        .style("opacity", opacity);
  };
}

''')

In [30]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

js_text = js_text_template.substitute({'graphdiv': 'graph-div'})

HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))